In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
!pip install tf2cv tensorflow>=2.0.0

In [2]:
from tensorflow.python.client import device_lib
dev=device_lib.list_local_devices()
dev[len(dev)-1]

name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14912199066
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6312810224809886538
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"

In [0]:
#global consts
IMG_SIZE=128
BATCH_SIZE=32
DROP_OUT=0.2
FOLD=0
TRAIN_DIR = './train/'
TRAIN_IMG_DIR=TRAIN_DIR+str(IMG_SIZE)
EPOCHS = 10
STATS = (0.0692, 0.2051)
NET_NAME='debug'
INPUT_DIR ='/content/drive/My Drive/kaggle/bengali/input/'
OUTPUT_DIR = '/content/drive/My Drive/kaggle/bengali/output/'
IMPORT_DIR = '/content/drive/My Drive/kaggle/bengali/nbs/'
OUTPUT_SUBDIR = ''
DATASET_224='224x224-bengali.zip'
DATASET_128='grapheme-imgs-128x128.zip'
TRAIN_WITH_FOLD = 'train_with_fold.csv'
PRETRAINED = False
PRETRAINED_WEIGHTS = 'w_final_checkp_debug_eps10_iniep10_sz128_bs32_do0.2_Adam_IS_fold0.h5'
LOG_FILE = OUTPUT_DIR+'log_'+NET_NAME+'.csv'
INITIAL_EPOCH=0
HEAD_WD=1e-3
BACKBONE_WD = 1e-4

if IMG_SIZE == 128: DATASET=DATASET_128
if IMG_SIZE == 224: DATASET=DATASET_224


#cosine anneling consts
LR_MAX = 0.01
LR_MIN = 0.001
T_MAX = 10

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append(IMPORT_DIR+'py/models/')
sys.path.append(IMPORT_DIR+'py/utils/')
sys.path.append(IMPORT_DIR+'py/callbacks/')
sys.path.append(IMPORT_DIR+'py/opts/')

In [0]:
#import local modules
from mish import Mish,mish
from generators import *
from kaggle_metric import *
from cosine import CosineAnnealingScheduler
from kagglevalidation import KaggleValidation
from lookahead import Lookahead
from se_resnext50 import build_se_resnext50_model
from md121v2 import build_md121_v2_model
from losses import categorical_focal_loss


#import external modules
import numpy as np 
import pandas as pd 

import os
import errno

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.utils import class_weight

In [0]:
try:
  os.mkdir(TRAIN_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(TRAIN_DIR+' already exists')
    else:
        raise
try:
  os.mkdir(TRAIN_IMG_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(TRAIN_IMG_DIR+' already exists')
    else:
        raise

os.system('cp '+ '"'+INPUT_DIR+DATASET+'" ' + TRAIN_DIR)
os.system('cp '+ '"'+INPUT_DIR+TRAIN_WITH_FOLD+'" ' + TRAIN_DIR)

0

In [0]:
if PRETRAINED: os.system('cp '+ '"'+OUTPUT_DIR+OUTPUT_SUBDIR+PRETRAINED_WEIGHTS+'" '+TRAIN_DIR)

In [0]:
os.system('unzip -q '+TRAIN_DIR+DATASET+ ' -d '+ TRAIN_IMG_DIR)

0

In [0]:
!ls train

128  grapheme-imgs-128x128.zip	train_with_fold.csv


In [0]:
dataset_np = pd.read_csv(TRAIN_DIR+TRAIN_WITH_FOLD).to_numpy()
dataset_np[:,0]+='.png'
dataset_m = dataset_np.shape[0]


valid_m = np.where(dataset_np[:,6]==FOLD)[0].shape[0]
train_m = dataset_m-valid_m

fold_train_inds = np.where(dataset_np[:,6] != FOLD)
fold_valid_inds = np.where(dataset_np[:,6] == FOLD)

train_np = dataset_np[fold_train_inds]
valid_np = dataset_np[fold_valid_inds]

train_df = pd.DataFrame(train_np)
valid_df = pd.DataFrame(valid_np)

train_df.rename(columns={0:'filename',1:'root_class',2:'vowel_class',3:'cons_class', 4:'grapheme'}, inplace=True)
valid_df.rename(columns={0:'filename',1:'root_class',2:'vowel_class',3:'cons_class', 4:'grapheme'}, inplace=True)

train_df.drop(columns=[5,6], inplace=True)
valid_df.drop(columns=[5,6], inplace=True)

In [0]:
cons_unique=np.unique(train_np[:,3])
cons_y_train = train_np[:,3]
vowel_unique=np.unique(train_np[:,2])
vowel_y_train = train_np[:,2]
root_unique =np.unique(train_np[:,1])
root_y_train = train_np[:,1]

cons_class_weights = class_weight.compute_class_weight('balanced',
                                                 cons_unique,
                                                 cons_y_train)

vowel_class_weights = class_weight.compute_class_weight('balanced',
                                                 vowel_unique,
                                                 vowel_y_train)

root_class_weights = class_weight.compute_class_weight('balanced',
                                                 root_unique,
                                                 root_y_train)
cons_cw_dict=dict(enumerate(cons_class_weights))
vowel_cw_dict=dict(enumerate(vowel_class_weights))
root_cw_dict=dict(enumerate(root_class_weights))

model_cw={}
model_cw['root']=root_cw_dict
model_cw['vowel']=vowel_cw_dict
model_cw['consonant']=cons_cw_dict

In [0]:
hard_roots=np.array([ 20,  32,  54,  60,  61,  62,  63,  67,  84,  85,  86, 104, 116,135, 140, 144, 145, 152, 154, 162])

In [0]:
model =  build_se_resnext50_model(drop_out=DROP_OUT)

In [0]:
if 0: model.summary()

In [0]:
if 0:
  from tensorflow.keras.utils import plot_model
  plot_model(model, to_file=OUTPUT_DIR+'model3.png',show_shapes=True)

In [0]:
if 0:
  for (n, layer) in enumerate(model.layers[1].layers):
    if 'activation' in layer.get_config() and layer.get_config()['activation'] == 'relu':
      print('replacing #{}: {}, {}'.format(n, layer, layer.activation))
      layer.activation = Mish(mish)
      print('-> {}'.format(layer.activation))

In [0]:
import tempfile

def apply_modifications(model, custom_objects=None):
    """Applies modifications to the model layers to create a new Graph. For example, simply changing
    `model.layers[idx].activation = new activation` does not change the graph. The entire graph needs to be updated
    with modified inbound and outbound tensors because of change in layer building function.
    Args:
        model: The `keras.models.Model` instance.
    Returns:
        The modified model with changes applied. Does not mutate the original `model`.
        reference: https://github.com/raghakot/keras-vis
    """
    
    # The strategy is to save the modified model and load it back. This is done because setting the activation
    # in a Keras layer doesnt actually change the graph. We have to iterate the entire graph and change the
    # layer inbound and outbound nodes with modified tensors. This is doubly complicated in Keras 2.x since
    # multiple inbound and outbound nodes are allowed with the Graph API.
    model_path = os.path.join(tempfile.gettempdir(), next(tempfile._get_candidate_names()) + '.h5')
    try:
        model.save(model_path)
        return load_model(model_path, custom_objects=custom_objects)
    finally:
        os.remove(model_path)

In [0]:
if 0:
  model = apply_modifications(model, custom_objects={'mish':Mish(mish)})

In [0]:
loss_dict={'root':     categorical_focal_loss(),
           'vowel':    'categorical_crossentropy',
           'consonant':'categorical_crossentropy'}

if 0: opt = Lookahead(tf.keras.optimizers.SGD(learning_rate=LR_MAX, momentum=0.9,nesterov=True))

if 1: opt=tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt, loss=loss_dict, loss_weights=[2.0,1.0,1.0])

In [0]:
cutout_datagen = ImageDataGenerator(rotation_range = 20,
                                   shear_range = 0.2,
                                   zoom_range = 0.1,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   preprocessing_function=get_random_eraser(s_l=0.05,s_h=0.1,r_1=0.8,r_2=1.2,v_l=0,v_h=0))

mcm_datagen = ImageDataGenerator(rotation_range = 20,
                                   shear_range = 0.2,
                                   zoom_range = 0.1,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1)

In [0]:
columns=["root_class","vowel_class", "cons_class"]

mcm_generator1 = mcm_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")

mcm_generator2 = mcm_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")


cutout_generator = cutout_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")


Found 160672 validated image filenames.
Found 160672 validated image filenames.
Found 160672 validated image filenames.


In [0]:
mixup_datagen=mixup_data_gen(aux_data_gen(mcm_generator1),aux_data_gen(mcm_generator2))
cutout_datagen=aux_data_gen(cutout_generator)
h_datagen=hybrid_data_gen(mixup_datagen,cutout_datagen)
h2_datagen=h_generator(mcm_generator1,mcm_generator2,cutout_generator)
hmcm_datagen=hmcm_generator(mcm_generator1,mcm_generator2)

In [0]:
if 0: 
  batch_x,y = next(h2_datagen)
  import matplotlib.pyplot as plt
  fig, axs = plt.subplots(1, 7, figsize=(30, 30))
  for i in range(7):
    axs[i].imshow(batch_x[i,:,:,0])
  plt.show()

In [0]:
TRAIN_DIR+PRETRAINED_WEIGHTS

'./train/w_final_checkp_debug_eps10_iniep10_sz128_bs32_do0.2_Adam_IS_fold0.h5'

In [0]:
if PRETRAINED: model.load_weights(TRAIN_DIR+PRETRAINED_WEIGHTS)

In [0]:
INITIAL_EPOCH

0

In [0]:
ca_shed = CosineAnnealingScheduler(T_max=T_MAX, eta_max=LR_MAX, eta_min=LR_MIN,verbose=1, initial_epoch=INITIAL_EPOCH)

suffix = NET_NAME+\
                '_eps'+str(EPOCHS)+\
                '_iniep'+str(INITIAL_EPOCH)+\
                '_sz'+str(IMG_SIZE)+\
                '_bs'+str(BATCH_SIZE)+\
                '_do'+str(DROP_OUT)+\
                '_'+model.optimizer.get_config()['name']+\
                '_IS_fold'+str(FOLD)


kv = KaggleValidation(valid_df,
                      batch_size=512,
                      vals_per_epoch=2,
                      train_steps=train_m//BATCH_SIZE+1,
                      logfile=OUTPUT_DIR+'log_kr_'+suffix+'.csv',
                      initial_epoch=INITIAL_EPOCH,
                      suffix = suffix,
                      train_img_dir=TRAIN_IMG_DIR,
                      output_dir=OUTPUT_DIR,
                      thresholds=[0.95,0.97,0.97])

history=model.fit(h2_datagen,
                  epochs=EPOCHS,
                  steps_per_epoch=train_m//BATCH_SIZE+1,
                  callbacks=[kv],
                  #class_weight=model_cw,
                  verbose=1)


model.save_weights(OUTPUT_DIR+'w_final_checkp_'+suffix+'.h5')
#model.save(OUTPUT_DIR+'m_'+suffix+'.h5')
#pd.DataFrame(history.history).to_csv(OUTPUT_DIR+'h_'+suffix+'.csv', index=False)

Train for 5022 steps
Epoch 1/10
2510/5022 [=============>................] - ETA: 10:33 - loss: 2.9649 - root_loss: 0.7367 - vowel_loss: 0.8786 - consonant_loss: 0.6129
batch 2510: validation...



[0:12:12] - kaggle:0.7135 - root:0.5220 -vowel:0.9006 - cons: 0.9095
5021/5022 [============================>.] - ETA: 0s - loss: 2.2789 - root_loss: 0.3299 - vowel_loss: 0.5535 - consonant_loss: 0.3798
batch 5021: validation...



[0:23:52] - kaggle:0.8310 - root:0.7440 -vowel:0.9236 - cons: 0.9124
5022/5022 [==============================] - 1433s 285ms/step - loss: 2.2787 - root_loss: 0.3299 - vowel_loss: 0.5535 - consonant_loss: 0.3798
Epoch 2/10
2510/5022 [=============>................] - ETA: 10:18 - loss: 1.3489 - root_loss: 0.2654 - vowel_loss: 0.4853 - consonant_loss: 0.3328
batch 2510: validation...



[0:35:44] - kaggle:0.8826 - root:0.8323 -vowel:0.9317 - cons: 0.9341
5021/5022 [============================>.] - ETA: 0s - loss: 1.3067 - root_loss: 0.2441 - vowel_loss: 0.4616 - consonant_loss: 0.3151
batch 5021: validation...



[0:47:24] - kaggle:0.8947 - root:0.8464 -vowel:0.9457 - cons: 0.9403
5022/5022 [==============================] - 1413s 281ms/step - loss: 1.3066 - root_loss: 0.2440 - vowel_loss: 0.4615 - consonant_loss: 0.3150
Epoch 3/10
2510/5022 [=============>................] - ETA: 9:56 - loss: 1.1379 - root_loss: 0.2178 - vowel_loss: 0.4153 - consonant_loss: 0.2870
batch 2510: validation...


...


[1:57:07] - kaggle:0.9407 - root:0.9134 -vowel:0.9743 - cons: 0.9618
saving weights w_best_[v 0.97427] best combined kaggle recall: 0.9610670052397455...
5022/5022 [==============================] - 1378s 274ms/step - loss: 0.9798 - root_loss: 0.1813 - vowel_loss: 0.3565 - consonant_loss: 0.2467
Epoch 6/10
2510/5022 [=============>................] - ETA: 10:00 - loss: 0.9501 - root_loss: 0.1783 - vowel_loss: 0.3518 - consonant_loss: 0.2417
batch 2510: validation...



[2:08:42] - kaggle:0.9331 - root:0.9016 -vowel:0.9718 - cons: 0.9573
5021/5022 [============================>.] - ETA: 0s - loss: 0.9398 - root_loss: 0.1730 - vowel_loss: 0.3441 - consonant_loss: 0.2396
batch 5021: validation...



[2:20:46] - kaggle:0.9086 - root:0.8666 -vowel:0.9645 - cons: 0.9368
5022/5022 [==============================] - 1418s 282ms/step - loss: 0.9397 - root_loss: 0.1730 - vowel_loss: 0.3440 - consonant_loss: 0.2395
Epoch 7/10
2510/5022 [=============>................] - ETA: 10:01 - loss: 0.9350 - root_loss: 0.1734 - vowel_loss: 0.3485 - consonant_loss: 0.2398
batch 2510: validation...



[2:32:18] - kaggle:0.9401 - root:0.9161 -vowel:0.9729 - cons: 0.9552
5021/5022 [============================>.] - ETA: 0s - loss: 0.9244 - root_loss: 0.1702 - vowel_loss: 0.3383 - consonant_loss: 0.2349
batch 5021: validation...



[2:43:37] - kaggle:0.9477 - root:0.9194 -vowel:0.9781 - cons: 0.9740
saving weights w_best_[v 0.97807][c 0.97398] best combined kaggle recall: 0.9630115000398162...
5022/5022 [==============================] - 1373s 273ms/step - loss: 0.9243 - root_loss: 0.1701 - vowel_loss: 0.3382 - consonant_loss: 0.2348
Epoch 8/10
  73/5022 [..............................] - ETA: 19:42 - loss: 1.0520 - root_loss: 0.1949 - vowel_loss: 0.3880 - consonant_loss: 0.2742

In [0]:
model.optimizer.get_config()

{'amsgrad': False,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'decay': 0.0,
 'epsilon': 1e-07,
 'learning_rate': 0.009140576,
 'name': 'Adam'}